In [ ]:
import keras
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
import json
import pickle

from keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Add, Embedding, Dropout, Flatten, LocallyConnected1D
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('drive/MyDrive/samples/tokenizer.json') as f:
  json_string = json.load(f)

# with open('drive/MyDrive/samples/tokenizer.json', 'rb') as handle:
#     tokenizer = pickle.load(handle)

tokenizer = keras.preprocessing.text.tokenizer_from_json(
    json_string
)
# keys = tokenizer.word_index.keys()

# drop = []
# drop_index = []

# for key in keys:
#   if tokenizer.word_index[key] >= 10000:
#     drop.append(key)
#     drop_index.append(tokenizer.word_index[key])

# print(len(drop))

# [tokenizer.word_index.pop(key) for key in drop]
# [tokenizer.index_word.pop(key) for key in drop_index]
# print(type(tokenizer.word_index))



dictionary_dim = len(tokenizer.word_index)

# print(dictionary_dim)

# tokenizer.word_index['<PAD>'] = 0
# tokenizer.index_word[0] = '<PAD>'

# tokenizer.word_index['<START>'] = dictionary_dim-1
# tokenizer.index_word[dictionary_dim-1] = '<START>'

# tokenizer.word_index['<END>'] = dictionary_dim
# tokenizer.index_word[dictionary_dim] = '<END>'





In [ ]:
tokenizer.word_index["\n"]

2

In [ ]:
# print(set(tokenizer.word_index.keys()) - set(tokenizer.index_word.values()))

In [ ]:
# print(dictionary_dim)
# print(len(tokenizer.index_word))
# print(len(tokenizer.word_index))

# diff = list(set(tokenizer.word_index.keys()) - set(tokenizer.index_word.values()))

In [ ]:
# for word in diff:
#   tokenizer.word_index.pop(word)

In [ ]:
embedding_dim = 128
max_input_len = 8
num_heads = 8
ffn_units = 4096
dropout_rate = 0

print(dictionary_dim)

# https://keras.io/examples/nlp/neural_machine_translation_with_transformer/
# def get_causal_attention_mask(inputs):
#     input_shape = tf.shape(inputs)
#     batch_size, sequence_length = input_shape[0], input_shape[1]
#     i = tf.range(sequence_length)[:, tf.newaxis]
#     j = tf.range(sequence_length)
#     mask = tf.cast(i >= j, dtype="int32")
#     mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
#     mult = tf.concat(
#         [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
#         axis=0,
#     )

#     if mask is not None:
#       padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
#       padding_mask = tf.minimum(padding_mask, tf.tile(mask, mult))

#     return [padding_mask, tf.tile(mask, mult)]
#

inp = Input(shape=(max_input_len,))
# target = Input(shape=(max_input_len,))

word_emb_1 = Embedding(input_dim=dictionary_dim, output_dim=embedding_dim)
word_emb_2 = Embedding(input_dim=dictionary_dim, output_dim=embedding_dim)

inp_emb = word_emb_1(inp)
# target_emb = word_emb_2(target)

### Postional encodings with help from https://machinelearningmastery.com/the-transformer-positional-encoding-layer-in-keras-part-2/
pos_emb = Embedding(input_dim=max_input_len, output_dim=embedding_dim)(tf.range(max_input_len))
###

inp_emb_sum = inp_emb + pos_emb
# target_emb_sum = target_emb + pos_emb

### START ENCODER

att = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(inp_emb_sum, inp_emb_sum)

dropout_1 = Dropout(dropout_rate)(att)

norm_1 = LayerNormalization()(inp_emb_sum + dropout_1)

dense_1 = Dense(units=ffn_units, activation="relu")(norm_1)

dense_2 = Dense(units=embedding_dim, activation="relu")(dense_1)

dropout_2 = Dropout(dropout_rate)(dense_2)

norm_2 = LayerNormalization()(norm_1 + dropout_2)

### END ENCODER

### START DECODER

# https://keras.io/examples/nlp/neural_machine_translation_with_transformer/

# padding_mask, causal_mask = get_causal_attention_mask(target_emb_sum)

# print(get_causal_attention_mask(target_emb_sum)[0][0].shape)
# print(get_causal_attention_mask(target_emb_sum)[1].shape)

# #

# print(causal_mask, padding_mask)

# # I don't understand the masking that's happening in att_2 and att_3 layers

# att_2 = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(query=target_emb_sum, 
#                                                                        value=target_emb_sum, key=target_emb_sum, attention_mask=get_causal_attention_mask(target_emb_sum)[1])

# norm_3 = LayerNormalization()(target_emb_sum + att_2)

# att_3 = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(query=norm_3, 
#                                                                        value=norm_2, key=norm_2, attention_mask=get_causal_attention_mask(target_emb_sum)[0][0])


flat_1 = Flatten()(norm_2)

funnel = Dense(2048, activation="relu")(flat_1)

out = Dense(10000 + 1, activation="softmax")(funnel)

### END DECODER

# local = LocallyConnected1D(128, 3)(norm_2)
# local_1 = LocallyConnected1D(128, 3)(local)

# flat = Flatten()(norm_2)

# out = Dense(units=dictionary_dim+1, activation="softmax")(flat)

model = Model(inp, out)

model.summary()




811174
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 8, 128)       103830272   ['input_3[0][0]']                
                                                                                                  
 tf.__operators__.add_6 (TFOpLa  (None, 8, 128)      0           ['embedding_6[0][0]']            
 mbda)                                                                                            
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 8, 128)      527488      ['tf.__operators__.add

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
              metrics=["sparse_categorical_accuracy", "sparse_top_k_categorical_accuracy"])

In [ ]:
# tokenizer = tf.keras.preprocessing.text.Tokenizer(
#     num_words=None,
#     filters="@",
#     split=None,
#     char_level=True,
#     lower=False
# )


In [ ]:
pkl = pd.read_pickle("drive/MyDrive/samples/train.pkl")


In [ ]:
pkl.head(50)

,path,samples,encoded
325956,Project_CodeNet/data/p02629/Python/s932742175....,"[n, =, int, (, input, (, ), ), \n, l, =, list,...","[23, 6, 17, 4, 16, 4, 3, 3, 2, 53, 6, 44, 4, 1..."
846024,Project_CodeNet/data/p02416/Python/s662498726....,"[while, True, :, \n, , x, =, input, (, ), ...","[77, 90, 7, 2, 1, 27, 6, 16, 4, 3, 2, 20, 27, ..."
266018,Project_CodeNet/data/p03150/Python/s061551833....,"[s, =, input, (, ), \n, import, re, \n, if, ""k...","[34, 6, 16, 4, 3, 2, 41, 298, 2, 20, 3254, 25,..."
828423,Project_CodeNet/data/p02659/Python/s855426226....,"[A, ,, B, =, input, (, ), ., split, (, ' ', ),...","[38, 5, 55, 6, 16, 4, 3, 11, 26, 4, 126, 3, 2,..."
524841,Project_CodeNet/data/p02400/Python/s711691697....,"[from, math, import, pi, \n, r, =, float, (, i...","[66, 84, 41, 260, 2, 58, 6, 139, 4, 16, 4, 3, ..."
72429,Project_CodeNet/data/p03463/Python/s886095411....,"[a, ,, b, ,, c, =, map, (, int, ,, input, (, )...","[22, 5, 32, 5, 42, 6, 33, 4, 17, 5, 16, 4, 3, ..."
1322644,Project_CodeNet/data/p03814/Python/s433791602....,"[# 入力, \n, s, =, input, (, ), \n, \n, # 処理, \n...","[45, 2, 34, 6, 16, 4, 3, 2, 2, 45, 2, 1, 6, 14..."
217675,Project_CodeNet/data/p04044/Python/s218572881....,"[n, ,, l, =, list, (, map, (, int, ,, input, (...","[23, 5, 53, 6, 44, 4, 33, 4, 17, 5, 16, 4, 3, ..."
22402,Project_CodeNet/data/p03337/Python/s595507867....,"[A, ,, B, =, list, (, map, (, int, ,, input, (...","[38, 5, 55, 6, 44, 4, 33, 4, 17, 5, 16, 4, 3, ..."
1239652,Project_CodeNet/data/p02697/Python/s124155220....,"[#ABC165-E Rotation Matching, \n, """"""\n各頂点を円形に...","[1, 2, 1, 2, 41, 51, 2, 81, 6, 51, 11, 78, 11,..."


In [ ]:
# pkl["encoded_2"] = pkl["samples"].apply(lambda x: [y[0] if len(y) != 0 else 1 for y in tokenizer.texts_to_sequences(x)])

In [ ]:
# j = 0

# for idx, row in pkl.iterrows():
#     j += 1

#     if j% 10000 == 0:
#       print(j)
#     cur_seq = pkl.loc[idx]["samples"]
#     #print(cur_seq)
#     tokenizer.fit_on_texts(cur_seq)

# j = 0

In [ ]:
# with open('drive/MyDrive/samples/char_tokenizer.pkl', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# model = load_model("drive/MyDrive/samples/transformer_model.h5")
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
              metrics=["sparse_categorical_accuracy", "sparse_top_k_categorical_accuracy"])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 8, 128)       103830272   ['input_3[0][0]']                
                                                                                                  
 tf.__operators__.add_6 (TFOpLa  (None, 8, 128)      0           ['embedding_6[0][0]']            
 mbda)                                                                                            
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 8, 128)      527488      ['tf.__operators__.add_6[0][0

In [ ]:
from keras.saving.saved_model import json_utils
mcp_save = ModelCheckpoint('drive/MyDrive/transformer_2.hdf5', save_best_only=True, monitor='val_sparse_categorical_accuracy', mode='max')
filename='drive/MyDrive/t_log.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)

seq_length = max_input_len

dataX = []

dataY = []
j = 0
#pkl = pkl.sample(frac=1)

pred_len = max_input_len

# for iteration in range(5000):

# pkl = pkl.sample(frac=.1)

dataX = []
target = []
dataY = []
for idx, row in pkl.iterrows():
  j += 1

  cur_seq = pkl.loc[idx]["encoded"]
  cur_seq = list(filter(lambda x: x > 2, cur_seq))
  #print(cur_seq)
  #tokenizer.fit_on_texts(cur_seq)
  # cur_seq = [tokenizer.word_index[word.strip()] for word in cur_seq]
  cur_seq = [0]*seq_length + [dictionary_dim - 1] + cur_seq + [dictionary_dim]
  for i in range(len(cur_seq) - seq_length - pred_len):
    cur = cur_seq[i:i+seq_length]
    cur = [0]*(max_input_len - len(cur)) + cur
    #print(len(cur))
    dataX.append(cur)
    #print(type(cur_seq[i+seq_length]))
    #print([[y] for y in cur_seq[i+seq_length: i+seq_length+pred_len]])
    # target.append([dictionary_dim-1]*1 + cur_seq[i+seq_length: i+seq_length+pred_len-1])
    dataY.append(cur_seq[i+seq_length+pred_len-1])
  if j % 1000 == 0:
    print("index is: ", j)

  if j % 1000000 == 0:
    print("FITTING MODEL ON BATCH...")

    x_t = np.array(dataX)
    y_t = np.array(dataY)

    print("DONE CONVERTING")

    

    # for num in range(1):
    #   idx = np.random.randint(0, dataX.shape[0], 10000)
    #   x_sample = dataX[idx]
    #   y_sample = dataY[idx]
    #   # target_sample = target[idx]

    #   # zeros = np.zeros((np.shape(dataY)[0],dictionary_dim))

    #   # for i in range(dictionary_dim):
    #   #   print(zeros.shape)
    #   #   zeros[dataY[i],i] = 1

    #   # dataY = zeros

    #   print(x_sample.shape)

    model.fit(x_t, y_t, epochs=5000, batch_size=512, steps_per_epoch = 1000, verbose=1, shuffle=True)

    # x_sample = []
    # y_sample = []
    #   # target_sample = []

    #   # print(dataX.shape)
    #   # print(target.shape)
    #   # print(dataY.shape)

    dataX = []
    dataY = []
    # target = []

    #seq_length = random.randint(4,max_input_len)

    # if j % 100000 == 0:  
    #   model.save("drive/MyDrive/samples/char_transformer.h5")




index is:  1000
index is:  2000
index is:  3000
index is:  4000
index is:  5000
index is:  6000
index is:  7000
index is:  8000
index is:  9000
index is:  10000
index is:  11000
index is:  12000
index is:  13000
index is:  14000
index is:  15000
index is:  16000
index is:  17000
index is:  18000
index is:  19000
index is:  20000
index is:  21000
index is:  22000
index is:  23000
index is:  24000
index is:  25000
index is:  26000
index is:  27000
index is:  28000
index is:  29000
index is:  30000
index is:  31000
index is:  32000
index is:  33000
index is:  34000
index is:  35000
index is:  36000
index is:  37000
index is:  38000
index is:  39000
index is:  40000
index is:  41000
index is:  42000
index is:  43000
index is:  44000
index is:  45000
index is:  46000
index is:  47000
index is:  48000
index is:  49000
index is:  50000
index is:  51000
index is:  52000
index is:  53000
index is:  54000
index is:  55000
index is:  56000
index is:  57000
index is:  58000
index is:  59000
index 

In [ ]:
history = model.fit(x_t, y_t, epochs=100, batch_size=512, verbose=1, 
                callbacks=[mcp_save, history_logger], shuffle=True)

Epoch 1/100
29249/60349 [=============>................] - ETA: 31:50 - loss: 3.1468 - sparse_categorical_accuracy: 0.3051 - sparse_top_k_categorical_accuracy: 0.5636

In [ ]:
1import tokenize
import io

test_sample = """if x == y"""

print(tokenizer.index_word[9998])

# target = np.array([9998] + [0]*(max_input_len - 1))

buf = io.StringIO(test_sample)

tokens = [tok.string.strip() for tok in tokenize.generate_tokens(buf.readline)]
print(tokens)

seq = tokenizer.texts_to_sequences(tokens)
inp = []

for x in seq:
  if len(x) != 0:
    inp.append(x[0])

if len(inp) > max_input_len:
  inp = inp[-10:-1]
  print(inp)

seq = np.array([0]*(max_input_len - len(inp)) + inp)
print(seq.shape)
seq = seq.reshape((1, seq.shape[0]))
# target = target.reshape((1, target.shape[0]))
print(seq.shape)
print(seq)
print(target)

pred = model.predict([seq, target])
pred = pred.flatten()

max = np.argmax(pred)
ind = np.argpartition(pred, -1)[-1:]


print(pred[ind])
print(max)
print(ind)

max = np.argmax(pred)
ind = np.argpartition(pred, -5)[-5:]
print(ind)
# idx = [[int(np.argmax(pred, axis=1))]]
# print(idx)

print(tokenizer.sequences_to_texts([[x] for x in ind]))

fact_list
['if', 'x', '==', 'y', '', '']
(8,)
(1, 8)
[[ 0  0  0  0 20 27 25 49]]
[]
[0.16272257]
2
[2]
[ 4  1  2 49 27]
['(', '<UNK>', '\n', 'y', 'x']


In [ ]:
model=load_model("drive/MyDrive/samples/transformer_model_3.h5")
model.summary()

max_input_len = 10

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, 10, 100)      1000000     ['input_8[0][0]']                
                                                                                                  
 tf.__operators__.add_22 (TFOpL  (None, 10, 100)     0           ['embedding_10[0][0]']           
 ambda)                                                                                           
                                                                                                  
 input_7 (InputLayer)           [(None, 10)]         0           []                         